# Data Preprocessing for Supply Chain Analysis

In [1]:
import pandas as pd

In [3]:
# Load datasets
supply_chain = pd.read_csv(r"C:\DA\Projects\Supply Chain & Sales Analytics System\Raw Data\DataCoSupplyChainDataset.csv", encoding="ISO-8859-1")
description = pd.read_csv(r"C:\DA\Projects\Supply Chain & Sales Analytics System\Raw Data\DescriptionDataCoSupplyChain.csv", encoding="ISO-8859-1")
access_logs = pd.read_csv(r"C:\DA\Projects\Supply Chain & Sales Analytics System\Raw Data\tokenized_access_logs.csv", encoding="ISO-8859-1")

In [4]:
# Standardizing column names (lowercase & replacing spaces with underscores)
def standardize_columns(df):
    df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace(r"[()]", "", regex=True)
    return df

supply_chain = standardize_columns(supply_chain)
description = standardize_columns(description)
access_logs = standardize_columns(access_logs)

In [10]:
# Display basic info about each dataset
print ("Info about supply chain:")
supply_chain_info = supply_chain.info()
print ("\nInfo about supply chain description:")
description_info = description.info()
print ("\nInfo about access logs:")
access_logs_info = access_logs.info()

supply_chain_info, description_info, access_logs_info

Info about supply chain:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   type                         180519 non-null  object 
 1   days_for_shipping_real       180519 non-null  int64  
 2   days_for_shipment_scheduled  180519 non-null  int64  
 3   benefit_per_order            180519 non-null  float64
 4   sales_per_customer           180519 non-null  float64
 5   delivery_status              180519 non-null  object 
 6   late_delivery_risk           180519 non-null  int64  
 7   category_id                  180519 non-null  int64  
 8   category_name                180519 non-null  object 
 9   customer_city                180519 non-null  object 
 10  customer_country             180519 non-null  object 
 11  customer_email               180519 non-null  object 
 12  customer_fname               1805

(None, None, None)

In [15]:
# Checking for missing values
print("\nMissing Values in Supply Chain Dataset:")
print(supply_chain.isnull().sum())
print("\nMissing Values in Description Dataset:")
print(description.isnull().sum())
print("\nMissing Values in Access Logs Dataset:")
print(access_logs.isnull().sum())


Missing Values in Supply Chain Dataset:
type                           0
days_for_shipping_real         0
days_for_shipment_scheduled    0
benefit_per_order              0
sales_per_customer             0
delivery_status                0
late_delivery_risk             0
category_id                    0
category_name                  0
customer_city                  0
customer_country               0
customer_email                 0
customer_fname                 0
customer_id                    0
customer_lname                 8
customer_password              0
customer_segment               0
customer_state                 0
customer_street                0
customer_zipcode               0
department_id                  0
department_name                0
latitude                       0
longitude                      0
market                         0
order_city                     0
order_country                  0
order_customer_id              0
order_date_dateorders          0
or

In [16]:
# Checking for duplicate entries
print("\nDuplicate Rows in Supply Chain Dataset:", supply_chain.duplicated().sum())
print("Duplicate Rows in Description Dataset:", description.duplicated().sum())
print("Duplicate Rows in Access Logs Dataset:", access_logs.duplicated().sum())


Duplicate Rows in Supply Chain Dataset: 0
Duplicate Rows in Description Dataset: 0
Duplicate Rows in Access Logs Dataset: 3249


In [12]:
# Convert date columns to datetime format
supply_chain["order_date_dateorders"] = pd.to_datetime(supply_chain["order_date_dateorders"], errors="coerce")
supply_chain["shipping_date_dateorders"] = pd.to_datetime(supply_chain["shipping_date_dateorders"], errors="coerce")

# Convert numeric columns with incorrect data types
supply_chain["customer_zipcode"] = supply_chain["customer_zipcode"].fillna(-1).astype(int)
supply_chain["order_zipcode"] = supply_chain["order_zipcode"].fillna(-1).astype(int)

# Drop the 'Product Description' column since it contains no data
supply_chain.drop(columns=["product_description"], inplace=True, errors="ignore")


In [18]:
# Convert 'Date' column to datetime format
access_logs["date"] = pd.to_datetime(access_logs["date"], errors="coerce")

# Convert 'Hour' column to integer (if not already)
access_logs["hour"] = access_logs["hour"].astype(int)

access_logs = access_logs.drop_duplicates()


In [19]:
# Display basic info about each dataset
print ("Info about supply chain:")
supply_chain_info = supply_chain.info()
print ("\nInfo about supply chain description:")
description_info = description.info()
print ("\nInfo about access logs:")
access_logs_info = access_logs.info()

Info about supply chain:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   type                         180519 non-null  object        
 1   days_for_shipping_real       180519 non-null  int64         
 2   days_for_shipment_scheduled  180519 non-null  int64         
 3   benefit_per_order            180519 non-null  float64       
 4   sales_per_customer           180519 non-null  float64       
 5   delivery_status              180519 non-null  object        
 6   late_delivery_risk           180519 non-null  int64         
 7   category_id                  180519 non-null  int64         
 8   category_name                180519 non-null  object        
 9   customer_city                180519 non-null  object        
 10  customer_country             180519 non-null  object        
 11  c

In [20]:
# Checking for duplicate entries
print("\nDuplicate Rows in Supply Chain Dataset:", supply_chain.duplicated().sum())
print("Duplicate Rows in Description Dataset:", description.duplicated().sum())
print("Duplicate Rows in Access Logs Dataset:", access_logs.duplicated().sum())


Duplicate Rows in Supply Chain Dataset: 0
Duplicate Rows in Description Dataset: 0
Duplicate Rows in Access Logs Dataset: 0


In [21]:
# Export cleaned datasets to CSV for SQL ingestion
supply_chain.to_csv("cleaned_supply_chain.csv", index=False)
access_logs.to_csv("cleaned_access_logs.csv", index=False)
description.to_csv("cleaned_description.csv", index=False)

print("Preprocessed data saved successfully.")

Preprocessed data saved successfully.


In [ ]:
# Extracting tables from the cleaned data
customers = supply_chain[[
    "order_customer_id", "customer_fname", "customer_lname",
    "customer_email", "customer_segment", "order_city",
    "order_state", "order_country", "order_zipcode"
]].drop_duplicates().rename(columns={"order_customer_id": "customer_id"})

In [ ]:
orders = supply_chain[[
    "order_id", "order_date_dateorders", "order_customer_id",
    "order_status", "order_region", "order_state",
    "sales", "order_profit_per_order", "order_item_total",
    "order_zipcode"
]].drop_duplicates().rename(columns={"order_date_dateorders": "order_date"})

In [ ]:
order_items = supply_chain[[
    "order_item_id", "order_id", "order_item_cardprod_id",
    "order_item_quantity", "order_item_product_price",
    "order_item_discount", "order_item_discount_rate",
    "order_item_profit_ratio"
]].drop_duplicates().rename(columns={"order_item_cardprod_id": "product_id"})

In [ ]:
products = supply_chain[[
    "order_item_cardprod_id", "product_name",
    "product_category_id", "order_item_product_price"
]].drop_duplicates().rename(columns={"order_item_cardprod_id": "product_id",
                                     "order_item_product_price": "product_price"})

In [ ]:
categories = supply_chain[[
    "product_category_id", "category_name",
    "department_id"
]].drop_duplicates()

In [34]:
departments = supply_chain[[
    "department_id", "department_name"
]].drop_duplicates()


In [36]:
shipping = supply_chain[[
    "order_id", "shipping_mode", "shipping_date_dateorders",
    "days_for_shipping_real", "days_for_shipment_scheduled",
    "delivery_status", "late_delivery_risk"
]].drop_duplicates().reset_index(drop=True).rename(columns={"index": "shipping_id"})


In [39]:
website_traffic = supply_chain[[
    "product_card_id",
    "product_category_id",
    "order_date_dateorders",
    "order_region",
    "order_state"
]].drop_duplicates().rename(columns={"order_date_dateorders": "date", "order_region": "department"})


In [40]:
returns = supply_chain[[
    "order_id", "order_item_cardprod_id",
    "order_item_discount"
]].drop_duplicates().rename(columns={"order_item_cardprod_id": "product_id",
                                     "order_item_discount": "refund_amount"}).reset_index(drop=True).reset_index().rename(columns={"index": "return_id"})


In [41]:
discounts = supply_chain[[
    "order_item_cardprod_id", "order_id",
    "order_item_discount_rate", "order_item_discount"
]].drop_duplicates().rename(columns={"order_item_cardprod_id": "product_id",
                                     "order_item_discount_rate": "discount_percentage",
                                     "order_item_discount": "discount_amount"}).reset_index(drop=True).reset_index().rename(columns={"index": "discount_id"})


In [46]:
# Save all tables as CSV files
tables = {
    "customers": customers,
    "orders": orders,
    "order_items": order_items,
    "products": products,
    "categories": categories,
    "departments": departments,
    "shipping": shipping,
    "returns": returns,
    "discounts": discounts,
    "website_traffic": website_traffic,
}

for name, df in tables.items():
    df.to_csv(f"{name}.csv", index=False)
    print(f"✅ Saved {name}.csv successfully!")


✅ Saved customers.csv successfully!
✅ Saved orders.csv successfully!
✅ Saved order_items.csv successfully!
✅ Saved products.csv successfully!
✅ Saved categories.csv successfully!
✅ Saved departments.csv successfully!
✅ Saved shipping.csv successfully!
✅ Saved returns.csv successfully!
✅ Saved discounts.csv successfully!
✅ Saved website_traffic.csv successfully!
